## Mounting drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Downloading GLUE data

In [0]:
import os
import sys
import shutil
import argparse
import tempfile
import urllib.request
import zipfile

## All BERT operations

In [0]:
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_repo'...
remote: Enumerating objects: 333, done.
remote: Total 333 (delta 0), reused 0 (delta 0), pack-reused 333
Receiving objects: 100% (333/333), 282.45 KiB | 3.87 MiB/s, done.
Resolving deltas: 100% (183/183), done.



In [0]:
!test -d vocab_repo || git clone https://github.com/kwonmha/bert-vocab-builder vocab_repo
if not 'vocab_repo' in sys.path:
  sys.path += ['vocab_repo']

Cloning into 'vocab_repo'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 88 (delta 46), reused 60 (delta 20), pack-reused 0
Unpacking objects: 100% (88/88), done.


## Parameters and hyper-parameters of BERT

In [0]:
import zipfile

In [0]:
with zipfile.ZipFile('/content/gdrive/My Drive/Colab Notebooks/RnD/weights/uncased_L-12_H-768_A-12.zip', 'r') as f:
  f.extractall('/content/BERT')

## Create vocab.txt

In [0]:
!python /content/vocab_repo/subword_builder.py \
--corpus_filepattern=/content/sample_data/training_dataset.txt \
--output_filename=/content/vocab_repo/deep_vocab.txt \
--split_on_newlines=True \
--min_count=5


[]
read all files

W0909 10:00:48.923957 140494984869760 deprecation_wrapper.py:119] From /content/vocab_repo/text_encoder.py:588: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Iteration 0
I0909 10:00:48.924238 140494984869760 text_encoder.py:588] Iteration 0
INFO:tensorflow:vocab_size = 61
I0909 10:00:48.932613 140494984869760 text_encoder.py:660] vocab_size = 61
INFO:tensorflow:Iteration 1
I0909 10:00:48.932847 140494984869760 text_encoder.py:588] Iteration 1
INFO:tensorflow:vocab_size = 61
I0909 10:00:48.938497 140494984869760 text_encoder.py:660] vocab_size = 61
INFO:tensorflow:Iteration 2
I0909 10:00:48.938714 140494984869760 text_encoder.py:588] Iteration 2
INFO:tensorflow:vocab_size = 61
I0909 10:00:48.944035 140494984869760 text_encoder.py:660] vocab_size = 61
INFO:tensorflow:Iteration 3
I0909 10:00:48.944238 140494984869760 text_encoder.py:588] Iteration 3
INFO:tensorflow:vocab_size = 61
I0909 10:00:48.949329 1404949848

## Finetuning


In [0]:
!export BERT_BASE_DIR=/content/BERT/uncased_L-12_H-768_A-12
!export GLUE_DIR=/content/glue_data

In [0]:
! python /content/bert_repo/run_classifier.py \
  --task_name=MRPC \
  --do_train=true \
  --do_eval=true \
  --data_dir=/content/glue_data/QQP \
  --vocab_file=/content/BERT/uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=/content/BERT/uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=/content/BERT/uncased_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=32 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \
  --output_dir=/tmp/qqp_output/




W0926 12:18:05.235652 140626898892672 deprecation_wrapper.py:119] From /content/bert_repo/run_classifier.py:784: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0926 12:18:05.235948 140626898892672 deprecation_wrapper.py:119] From /content/bert_repo/run_classifier.py:784: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0926 12:18:05.236369 140626898892672 deprecation_wrapper.py:119] From /content/bert_repo/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0926 12:18:05.237106 140626898892672 deprecation_wrapper.py:119] From /content/bert_repo/run_classifier.py:808: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

I0926 12:18:06.847019 140626898892672 utils.py:141] NumExpr defaulting to 2 threads.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * ht

In [0]:
!python /content/bert_repo/run_pretraining.py \
  --input_file=/tmp/tf_examples.tfrecord \
  --output_dir=/content/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=/content/BERT/uncased_L-12_H-768_A-12/bert_config.json \
  --train_batch_size=32 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20 \
  --num_warmup_steps=10 \
  --learning_rate=2e-4




W0909 09:54:46.532331 140647264556928 deprecation_wrapper.py:119] From /content/bert_repo/run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0909 09:54:46.532544 140647264556928 deprecation_wrapper.py:119] From /content/bert_repo/run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0909 09:54:46.532708 140647264556928 deprecation_wrapper.py:119] From /content/bert_repo/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0909 09:54:46.533375 140647264556928 deprecation_wrapper.py:119] From /content/bert_repo/run_pretraining.py:414: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0909 09:54:46.533866 140647264556928 deprecation_wrapper.py:119] From /content/bert_repo/run_pretraining.py:418: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.




## Extracting features

In [0]:
! python /content/bert_repo/extract_features.py \
  --input_file=/content/sample_data/training_data.txt \
  --output_file=/tmp/output.jsonl \
  --vocab_file=/content/vocab_repo/deep_vocab.txt \
  --bert_config_file=/content/BERT/uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=/content/BERT/uncased_L-12_H-768_A-12/bert_model.ckpt \
  --layers=-1 \
  --max_seq_length=128 \
  --batch_size=8



W0909 11:24:48.356233 139703173781376 deprecation_wrapper.py:119] From /content/bert_repo/extract_features.py:344: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0909 11:24:48.356512 139703173781376 deprecation_wrapper.py:119] From /content/bert_repo/extract_features.py:344: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0909 11:24:48.356690 139703173781376 deprecation_wrapper.py:119] From /content/bert_repo/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

I0909 11:24:49.069122 139703173781376 utils.py:141] NumExpr defaulting to 2 threads.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend

# Extra

## Jsonl to embedding

In [0]:
import json
import numpy as np

In [0]:
def get_bio_bert_embedding(sentence):
  
  outF = open("/input.txt", "w")
  outF.write(sentence)
  outF.close()
  
  os.system('python3 /content/bert_repo/extract_features.py \
   --input_file=/content/sample_data/deeplearning.txt \
   --vocab_file=/content/BERT/uncased_L-12_H-768_A-12/vocab.txt \
   --bert_config_file=/content/biobert/biobert_pubmed/bert_config.json \
   --init_checkpoint=/content/pretraining_output/model.ckpt-20\
   --output_file=/tmp/output.jsonl')

  d = []
  with open('/content/gdrive/My Drive/Colab Notebooks/RnD/weights/output.jsonl') as f:
      for line in f:
          d.append(json.loads(line))
      
  sentence_vector = np.zeros(768).tolist()
  for i in range (1, len(d[0]['features'])-1):
    sentence_vector = sentence_vector + d[0]['features'][i]['layers'][0]['values']
  print(d[0]['features'])
  number_of_tokens = len(d[0]['features']) - 2
  print(number_of_tokens)
  for elem in sentence_vector:
    elem = elem/number_of_tokens
  return sentence_vector

In [0]:
sent_vector = get_bio_bert_embedding('I am good boy')

In [0]:
print(sent_vector)

## Finetuning

In [0]:
!export BERT_BASE_DIR=/content/BERT/uncased_L-12_H-768_A-12
!export GLUE_DIR=/content/glue_data
!python /content/bert_repo/run_classifier.py\
  --task_name=MRPC \
  --do_train=True \
  --do_eval=True \
  --data_dir=/content/glue_data/CoLA \
  --vocab_file=/content/BERT/uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=/content/BERT/uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=/content/BERT/uncased_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=32 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \
  --output_dir=/content/results/CoLA_output/

In [0]:
!export BERT_BASE_DIR=/content/BERT/uncased_L-12_H-768_A-12
!export GLUE_DIR=/content/glue_data
!export TRAINED_CLASSIFIER=/content/results/CoLA_output

!python /content/bert_repo/optimization.py \
  --task_name=CoLA \
  --do_predict=True \
  --data_dir=/content/glue_data/CoLA \
  --vocab_file=/content/BERT/uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=/content/BERT/uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=/content/BERT/uncased_L-12_H-768_A-12/bert_model.ckpt\
  --max_seq_length=128 \
  --output_dir=/content/results_cola

In [0]:
import tensorflow as tf

for example in tf.python_io.tf_record_iterator("/content/results/predict.tf_record"):
    print(tf.train.Example.FromString(example))

In [0]:
import json

In [0]:
embedding = []
with open('/tmp/output.jsonl') as f:
    for line in f:
        embedding.append(json.loads(line))

In [0]:
print(len(embedding))

In [0]:
print(embedding)

## GLUE data

In [0]:
""" Script for downloading all GLUE data.
Example usage:
    python download_glue_data.py --data_dir data --tasks all
Note: for legal reasons, we are unable to host MRPC.
You can either use the version hosted by the SentEval team, which is already tokenized,
or you can download the original data from:
https://download.microsoft.com/download/D/4/6/D46FF87A-F6B9-4252-AA8B-3604ED519838/MSRParaphraseCorpus.msi  # noqa
and extract the data from it manually.
For Windows users, you can run the .msi file. For Mac and Linux users, consider an external library
such as 'cabextract' (see below for an example). You should then rename and place specific files in
a folder (see below for an example).
mkdir MRPCA
cabextract MSRParaphraseCorpus.msi -d MRPC
cat MRPC/_2DEC3DBE877E4DB192D17C0256E90F1D | tr -d $'\r' > MRPC/msr_paraphrase_train.txt
cat MRPC/_D7B391F9EAFF4B1B8BCE8F21B20B1B61 | tr -d $'\r' > MRPC/msr_paraphrase_test.txt
rm MRPC/_*
rm MSRParaphraseCorpus.msi
"""



TASKS = ["CoLA", "SST", "MRPC", "QQP", "STS", "MNLI", "SNLI", "QNLI", "RTE", "WNLI", "diagnostic"]
TASK2PATH = {
    "CoLA": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FCoLA.zip?alt=media&token=46d5e637-3411-4188-bc44-5809b5bfb5f4",  # noqa
    "SST": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8",  # noqa
    "MRPC": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc",  # noqa
    "QQP": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQQP-clean.zip?alt=media&token=11a647cb-ecd3-49c9-9d31-79f8ca8fe277",  # noqa
    "STS": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSTS-B.zip?alt=media&token=bddb94a7-8706-4e0d-a694-1109e12273b5",  # noqa
    "MNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FMNLI.zip?alt=media&token=50329ea1-e339-40e2-809c-10c40afff3ce",  # noqa
    "SNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSNLI.zip?alt=media&token=4afcfbb2-ff0c-4b2d-a09a-dbf07926f4df",  # noqa
    "QNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQNLIv2.zip?alt=media&token=6fdcf570-0fc5-4631-8456-9505272d1601",  # noqa
    "RTE": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FRTE.zip?alt=media&token=5efa7e85-a0bb-4f19-8ea2-9e1840f077fb",  # noqa
    "WNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FWNLI.zip?alt=media&token=068ad0a0-ded7-4bd7-99a5-5e00222e0faf",  # noqa
    "diagnostic": ["https://storage.googleapis.com/mtl-sentence-representations.appspot.com/tsvsWithoutLabels%2FAX.tsv?GoogleAccessId=firebase-adminsdk-0khhl@mtl-sentence-representations.iam.gserviceaccount.com&Expires=2498860800&Signature=DuQ2CSPt2Yfre0C%2BiISrVYrIFaZH1Lc7hBVZDD4ZyR7fZYOMNOUGpi8QxBmTNOrNPjR3z1cggo7WXFfrgECP6FBJSsURv8Ybrue8Ypt%2FTPxbuJ0Xc2FhDi%2BarnecCBFO77RSbfuz%2Bs95hRrYhTnByqu3U%2FYZPaj3tZt5QdfpH2IUROY8LiBXoXS46LE%2FgOQc%2FKN%2BA9SoscRDYsnxHfG0IjXGwHN%2Bf88q6hOmAxeNPx6moDulUF6XMUAaXCSFU%2BnRO2RDL9CapWxj%2BDl7syNyHhB7987hZ80B%2FwFkQ3MEs8auvt5XW1%2Bd4aCU7ytgM69r8JDCwibfhZxpaa4gd50QXQ%3D%3D",  # noqa
        "https://www.dropbox.com/s/ju7d95ifb072q9f/diagnostic-full.tsv?dl=1",],}

MRPC_TRAIN = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt"
MRPC_TEST = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt"


def download_and_extract(task, data_dir):
    print("Downloading and extracting %s..." % task)
    data_file = "%s.zip" % task
    urllib.request.urlretrieve(TASK2PATH[task], data_file)
    with zipfile.ZipFile(data_file) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(data_file)
    print("\tCompleted!")


def format_mrpc(data_dir, path_to_data):
    print("Processing MRPC...")
    mrpc_dir = os.path.join(data_dir, "MRPC")
    if not os.path.isdir(mrpc_dir):
        os.mkdir(mrpc_dir)
    if path_to_data:
        mrpc_train_file = os.path.join(path_to_data, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(path_to_data, "msr_paraphrase_test.txt")
    else:
        print("Local MRPC data not specified, downloading data from %s" % MRPC_TRAIN)
        mrpc_train_file = os.path.join(mrpc_dir, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(mrpc_dir, "msr_paraphrase_test.txt")
        urllib.request.urlretrieve(MRPC_TRAIN, mrpc_train_file)
        urllib.request.urlretrieve(MRPC_TEST, mrpc_test_file)
    assert os.path.isfile(mrpc_train_file), "Train data not found at %s" % mrpc_train_file
    assert os.path.isfile(mrpc_test_file), "Test data not found at %s" % mrpc_test_file
    urllib.request.urlretrieve(TASK2PATH["MRPC"], os.path.join(mrpc_dir, "dev_ids.tsv"))

    dev_ids = []
    with open(os.path.join(mrpc_dir, "dev_ids.tsv"), encoding="utf8") as ids_fh:
        for row in ids_fh:
            dev_ids.append(row.strip().split("\t"))

    with open(mrpc_train_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "train.tsv"), "w", encoding="utf8"
    ) as train_fh, open(os.path.join(mrpc_dir, "dev.tsv"), "w", encoding="utf8") as dev_fh:
        header = data_fh.readline()
        train_fh.write(header)
        dev_fh.write(header)
        for row in data_fh:
            label, id1, id2, s1, s2 = row.strip().split("\t")
            if [id1, id2] in dev_ids:
                dev_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))
            else:
                train_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))

    with open(mrpc_test_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "test.tsv"), "w", encoding="utf8"
    ) as test_fh:
        header = data_fh.readline()
        test_fh.write("index\t#1 ID\t#2 ID\t#1 String\t#2 String\n")
        for idx, row in enumerate(data_fh):
            label, id1, id2, s1, s2 = row.strip().split("\t")
            test_fh.write("%d\t%s\t%s\t%s\t%s\n" % (idx, id1, id2, s1, s2))
    print("\tCompleted!")


def download_diagnostic(data_dir):
    print("Downloading and extracting diagnostic data...")
    if not os.path.isdir(os.path.join(data_dir, "MNLI")):
        os.mkdir(os.path.join(data_dir, "MNLI"))
    data_file = os.path.join(data_dir, "MNLI", "diagnostic.tsv")
    urllib.request.urlretrieve(TASK2PATH["diagnostic"][0], data_file)
    data_file = os.path.join(data_dir, "MNLI", "diagnostic-full.tsv")
    urllib.request.urlretrieve(TASK2PATH["diagnostic"][1], data_file)
    print("\tCompleted!")
    return


def get_tasks(task_names):
    task_names = task_names.split(",")
    if "all" in task_names:
        tasks = TASKS
    else:
        tasks = []
        for task_name in task_names:
            assert task_name in TASKS, "Task %s not found!" % task_name
            tasks.append(task_name)
        if "MNLI" in tasks and "diagnostic" not in tasks:
            tasks.append("diagnostic")

    return tasks


def main(arguments):
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--data_dir", help="directory to save data to", type=str, default="glue_data"
    )
    parser.add_argument(
        "--tasks",
        help="tasks to download data for as a comma separated string",
        type=str,
        default="all",
    )
    parser.add_argument(
        "--path_to_mrpc",
        help="path to directory containing extracted MRPC data, msr_paraphrase_train.txt and "
        "msr_paraphrase_text.txt",
        type=str,
        default="",
    )
    args = parser.parse_known_args(arguments)[0]

    if not os.path.isdir(args.data_dir):
        os.mkdir(args.data_dir)
    tasks = get_tasks(args.tasks)

    for task in tasks:
        if task == "MRPC":
            format_mrpc(args.data_dir, args.path_to_mrpc)
        elif task == "diagnostic":
            download_diagnostic(args.data_dir)
        else:
            download_and_extract(task, args.data_dir)


if __name__ == "__main__":
    sys.exit(main(sys.argv[1:]))

	Completed!
	Completed!
Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
